In [1]:
from __future__ import print_function
import numpy as np
import pandas as pd
import tensorflow as tf
import sys
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from keras.utils import np_utils


/Users/noahkasmanoff/anaconda/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:

def next_batch(num, data, labels):
    '''
    Return a total of `num` random samples and labels. 
    '''
    idx = np.arange(0 , len(data))
    np.random.shuffle(idx)
    idx = idx[:num]
    data_shuffle = [data[ i] for i in idx]
    labels_shuffle = [labels[ i] for i in idx]

    return np.asarray(data_shuffle), np.asarray(labels_shuffle)



In [3]:
data = pd.read_csv('cancer.dat',header=None)
data_array = data.values
#remove rows with missing vals
good_rows = np.invert(np.any(data_array=='?',1))
data = data_array[good_rows]
x= data[:,1:10].astype(np.float32)
y=data[:,10].astype(int)
#one hot encoding - have to use for y in neural nets, bc not automatically a classification problem
le = LabelEncoder()
le.fit(y)
y=le.transform(y)
y=np_utils.to_categorical(y)
(x_train,x_test,y_train,y_test) = train_test_split(x,y,test_size=.2)

In [4]:
np.shape(x_train)

(546, 9)

In [5]:
# Parameters
learning_rate = 0.1
num_steps = 500
batch_size = 500
display_step = 1

# Network Parameters
n_hidden_1 = 12 # 1st layer number of neurons
n_hidden_2 = 8 # 2nd layer number of neurons
num_input = 9 # MNIST data input (img shape: 28*28)
num_classes = 2 # MNIST total classes (0-9 digits)

# tf Graph input
X = tf.placeholder("float", [None, num_input])
Y = tf.placeholder("float", [None, num_classes])

In [6]:
# Store layers weight & bias
weights = {
    'h1': tf.Variable(tf.random_normal([num_input, n_hidden_1])),
    'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_hidden_2, num_classes]))
}
biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'b2': tf.Variable(tf.random_normal([n_hidden_2])),
    'out': tf.Variable(tf.random_normal([num_classes]))
}

In [7]:
# Create model
def neural_net(x):
    # Hidden fully connected layer with 256 neurons
    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
    relu_1 = tf.nn.relu(layer_1)
    # Hidden fully connected layer with 256 neurons
    layer_2 = tf.add(tf.matmul(relu_1, weights['h2']), biases['b2'])
    relu_2 = tf.nn.sigmoid(layer_2)

    # Output fully connected layer with a neuron for each class
    out_layer = tf.matmul(relu_2, weights['out']) + biases['out']
    return out_layer

In [8]:

# Construct model
logits = neural_net(X)

# Define loss and optimizer
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
    logits=logits, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)

# Evaluate model (with test logits, for dropout to be disabled)
correct_pred = tf.equal(tf.argmax(logits, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.



In [9]:
# Start training
with tf.Session() as sess:

    # Run the initializer
    sess.run(init)

    for step in range(1, num_steps+1):
        batch_x, batch_y = next_batch(batch_size,x_train,y_train)
        # Run optimization op (backprop)
        sess.run(train_op, feed_dict={X: batch_x, Y: batch_y})
        if step % display_step == 0 or step == 1:
            # Calculate batch loss and accuracy
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: batch_x,
                                                                 Y: batch_y})
            print("Step " + str(step) + ", Minibatch Loss= " + \
                  "{:.4f}".format(loss) + ", Training Accuracy= " + \
                  "{:.3f}".format(acc))

    print("Optimization Finished!")

    # Calculate accuracy for MNIST test images
    print("Testing Accuracy:", \
        sess.run(accuracy, feed_dict={X: x_test,
                                      Y: y_test}))

Step 1, Minibatch Loss= 0.6601, Training Accuracy= 0.656
Step 2, Minibatch Loss= 0.6219, Training Accuracy= 0.718
Step 3, Minibatch Loss= 0.7370, Training Accuracy= 0.618
Step 4, Minibatch Loss= 0.6405, Training Accuracy= 0.704
Step 5, Minibatch Loss= 0.5334, Training Accuracy= 0.852
Step 6, Minibatch Loss= 0.5431, Training Accuracy= 0.646
Step 7, Minibatch Loss= 0.5728, Training Accuracy= 0.658
Step 8, Minibatch Loss= 0.5719, Training Accuracy= 0.656
Step 9, Minibatch Loss= 0.5410, Training Accuracy= 0.658
Step 10, Minibatch Loss= 0.4666, Training Accuracy= 0.650
Step 11, Minibatch Loss= 0.3956, Training Accuracy= 0.914
Step 12, Minibatch Loss= 0.3984, Training Accuracy= 0.882
Step 13, Minibatch Loss= 0.3989, Training Accuracy= 0.860
Step 14, Minibatch Loss= 0.3937, Training Accuracy= 0.860
Step 15, Minibatch Loss= 0.3765, Training Accuracy= 0.866
Step 16, Minibatch Loss= 0.3430, Training Accuracy= 0.902
Step 17, Minibatch Loss= 0.3231, Training Accuracy= 0.910
Step 18, Minibatch Loss

Step 167, Minibatch Loss= 0.0523, Training Accuracy= 0.988
Step 168, Minibatch Loss= 0.0456, Training Accuracy= 0.988
Step 169, Minibatch Loss= 0.0510, Training Accuracy= 0.988
Step 170, Minibatch Loss= 0.0574, Training Accuracy= 0.986
Step 171, Minibatch Loss= 0.0504, Training Accuracy= 0.988
Step 172, Minibatch Loss= 0.0571, Training Accuracy= 0.986
Step 173, Minibatch Loss= 0.0646, Training Accuracy= 0.982
Step 174, Minibatch Loss= 0.0525, Training Accuracy= 0.988
Step 175, Minibatch Loss= 0.0625, Training Accuracy= 0.982
Step 176, Minibatch Loss= 0.0505, Training Accuracy= 0.990
Step 177, Minibatch Loss= 0.0608, Training Accuracy= 0.984
Step 178, Minibatch Loss= 0.0671, Training Accuracy= 0.976
Step 179, Minibatch Loss= 0.0552, Training Accuracy= 0.988
Step 180, Minibatch Loss= 0.0697, Training Accuracy= 0.978
Step 181, Minibatch Loss= 0.0685, Training Accuracy= 0.984
Step 182, Minibatch Loss= 0.0630, Training Accuracy= 0.982
Step 183, Minibatch Loss= 0.0803, Training Accuracy= 0.9

Step 331, Minibatch Loss= 0.0469, Training Accuracy= 0.990
Step 332, Minibatch Loss= 0.0469, Training Accuracy= 0.990
Step 333, Minibatch Loss= 0.0392, Training Accuracy= 0.992
Step 334, Minibatch Loss= 0.0465, Training Accuracy= 0.990
Step 335, Minibatch Loss= 0.0396, Training Accuracy= 0.992
Step 336, Minibatch Loss= 0.0391, Training Accuracy= 0.992
Step 337, Minibatch Loss= 0.0466, Training Accuracy= 0.990
Step 338, Minibatch Loss= 0.0467, Training Accuracy= 0.990
Step 339, Minibatch Loss= 0.0466, Training Accuracy= 0.990
Step 340, Minibatch Loss= 0.0397, Training Accuracy= 0.992
Step 341, Minibatch Loss= 0.0466, Training Accuracy= 0.990
Step 342, Minibatch Loss= 0.0459, Training Accuracy= 0.990
Step 343, Minibatch Loss= 0.0463, Training Accuracy= 0.990
Step 344, Minibatch Loss= 0.0461, Training Accuracy= 0.990
Step 345, Minibatch Loss= 0.0465, Training Accuracy= 0.990
Step 346, Minibatch Loss= 0.0395, Training Accuracy= 0.992
Step 347, Minibatch Loss= 0.0325, Training Accuracy= 0.9

Step 489, Minibatch Loss= 0.0521, Training Accuracy= 0.986
Step 490, Minibatch Loss= 0.0518, Training Accuracy= 0.986
Step 491, Minibatch Loss= 0.0448, Training Accuracy= 0.988
Step 492, Minibatch Loss= 0.0497, Training Accuracy= 0.986
Step 493, Minibatch Loss= 0.0495, Training Accuracy= 0.986
Step 494, Minibatch Loss= 0.0488, Training Accuracy= 0.986
Step 495, Minibatch Loss= 0.0490, Training Accuracy= 0.986
Step 496, Minibatch Loss= 0.0487, Training Accuracy= 0.986
Step 497, Minibatch Loss= 0.0482, Training Accuracy= 0.986
Step 498, Minibatch Loss= 0.0424, Training Accuracy= 0.988
Step 499, Minibatch Loss= 0.0469, Training Accuracy= 0.986
Step 500, Minibatch Loss= 0.0484, Training Accuracy= 0.986
Optimization Finished!
Testing Accuracy: 0.96350366
